In [2]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import plotly.express as px
from policyengine_core.charts import format_fig
import plotly.graph_objects as go


In [3]:
reform = Reform.from_dict({
  "gov.contrib.ubi_center.basic_income.amount.person.flat": {
    "2025-01-01.2025-12-31": 1160,
    "2026-01-01.2026-12-31": 1605,
    "2027-01-01.2027-12-31": 1686
  },
  "gov.contrib.ubi_center.basic_income.taxable": {
    "2024-01-01.2100-12-31": True
  }
}, country_id="us")

reform_taxable = Reform.from_dict({
  "gov.contrib.states.or.rebate.state_tax_exempt": {
    "2024-01-01.2100-12-31": True
  },
  "gov.contrib.ubi_center.basic_income.amount.person.flat": {
    "2025-01-01.2025-12-31": 1160,
    "2026-01-01.2026-12-31": 1605,
    "2027-01-01.2027-12-31": 1686
  },
  "gov.contrib.ubi_center.basic_income.taxable": {
    "2024-01-01.2100-12-31": True
  }
}, country_id="us")

In [4]:
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
reformed_taxable = Microsimulation(reform=reform_taxable)


In [5]:
def calculate_poverty_impact_by_age(baseline, reformed, year):
    state_codes = baseline.calc("state_code", map_to="person", period=year)
    age = baseline.calc("age", map_to="person", period=year)

    baseline_poverty = baseline.calc("in_poverty", map_to="person", period=year)
    reform_poverty = reformed.calc("in_poverty", map_to="person", period=year)

    results = {}
    age_groups = [
        (0, 18, "0-17"),
        (18, 65, "18-64"),
        (65, 200, "65+"),
        (0, 200, "Overall")
    ]

    for min_age, max_age, label in age_groups:
        if label == "Overall":
            mask = (state_codes == "OR")
        else:
            mask = (state_codes == "OR") & (age >= min_age) & (age < max_age)

        baseline_poverty_group = baseline_poverty[mask].mean()
        reform_poverty_group = reform_poverty[mask].mean()

        relative_poverty_reduction = (reform_poverty_group - baseline_poverty_group) / baseline_poverty_group
        results[label] = relative_poverty_reduction

    return results


In [6]:
# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame(columns=["year", "age_group", "relative_poverty_reduction", "relative_poverty_reduction_taxable"])


In [7]:
# Calculate for each year and add to the DataFrame
for year in range(2025, 2028):
    results_untaxed = calculate_poverty_impact_by_age(baseline, reformed, year)
    results_taxed = calculate_poverty_impact_by_age(baseline, reformed_taxable, year)

    for age_group in results_untaxed.keys():
        new_row = pd.DataFrame({
            "year": [year],
            "age_group": [age_group],
            "relative_poverty_reduction": [results_untaxed[age_group]],
            "relative_poverty_reduction_taxable": [results_taxed[age_group]]
        })
        all_results = pd.concat([all_results, new_row], ignore_index=True)


In [8]:
# Display the results
print(all_results)

    year age_group  relative_poverty_reduction  \
0   2025      0-17                   -0.392697   
1   2025     18-64                   -0.207817   
2   2025       65+                   -0.053603   
3   2025   Overall                   -0.216189   
4   2026      0-17                   -0.434428   
5   2026     18-64                   -0.235611   
6   2026       65+                   -0.050389   
7   2026   Overall                   -0.239394   
8   2027      0-17                   -0.423603   
9   2027     18-64                   -0.233850   
10  2027       65+                   -0.076262   
11  2027   Overall                   -0.241171   

    relative_poverty_reduction_taxable  
0                            -0.392697  
1                            -0.217906  
2                            -0.053603  
3                            -0.222731  
4                            -0.461682  
5                            -0.243466  
6                            -0.050389  
7                    

In [14]:
# Define colors for age groups
colors = {
    "0-17": "#003366",     # Dark blue
    "18-64": "#0066cc",    # Medium blue
    "65+": "#4d94ff",      # Light blue
    "Overall": "#99ccff",  # Very light blue
}

# Create the plot
fig = go.Figure()

# Store final y-values for label positioning
final_y_values = {}

# Add traces for each age group
for age_group in colors.keys():
    data = all_results[all_results['age_group'] == age_group]

    # Taxed scenario
    taxed_trace = go.Scatter(
        x=data['year'], 
        y=data['relative_poverty_reduction_taxable'],
        mode='lines',
        name=f'{age_group} (Taxed)',
        line=dict(color=colors[age_group], width=2),
        showlegend=False
    )
    fig.add_trace(taxed_trace)
    final_y_values[f"{age_group} (Taxed)"] = taxed_trace.y[-1]

    # Untaxed scenario
    untaxed_trace = go.Scatter(
        x=data['year'], 
        y=data['relative_poverty_reduction'],
        mode='lines',
        name=f'{age_group} (Untaxed)',
        line=dict(color=colors[age_group], width=2, dash='dash'),
        showlegend=False
    )
    fig.add_trace(untaxed_trace)
    final_y_values[f"{age_group} (Untaxed)"] = untaxed_trace.y[-1]

# Function to adjust label positions
def adjust_positions(positions, total_height, min_gap=0.02):
    sorted_items = sorted(positions.items(), key=lambda x: x[1])
    n = len(sorted_items)
    even_spread = total_height / (n + 1)
    
    adjusted = {}
    for i, (key, value) in enumerate(sorted_items):
        ideal_pos = (i + 1) * even_spread
        adjusted[key] = min(max(value, ideal_pos - even_spread/2), ideal_pos + even_spread/2)
    
    return adjusted

# Calculate total height of the plot
y_min, y_max = fig.layout.yaxis.range
total_height = y_max - y_min

# Adjust positions
adjusted_positions = adjust_positions(final_y_values, total_height)

# Add rotated text annotations
for label, y_pos in adjusted_positions.items():
    fig.add_annotation(
        x=2027.2,  # Slightly to the right of the last data point
        y=y_pos,
        text=label,
        textangle=90,  # Rotate 90 degrees to the left
        showarrow=False,
        xanchor="left",
        yanchor="middle",
        font=dict(size=10, color=colors[label.split()[0]])
    )

# Update layout
fig.update_layout(
    title='Oregon Rebate Impact on Poverty by Age Group and Federal Taxation Over Time',
    xaxis_title='Year',
    yaxis_title='Poverty Reduction (%)',
    height=600,
    width=800,
    margin=dict(r=150)  # Increase right margin for labels
)

# Update x-axis
fig.update_xaxes(
    tickvals=[2025, 2026, 2027],
    ticktext=["2025", "2026", "2027"],
    range=[2024.8, 2027.5]  # Extend x-axis slightly for labels
)

# Update y-axis
fig.update_yaxes(tickformat='.0%', range=[-0.5, 0])

# Update hover template
fig.update_traces(
    hovertemplate='Year: %{x}<br>Age Group: %{data.name}<br>Poverty Reduction: %{y:.2%}<extra></extra>'
)

fig = format_fig(fig)
fig.show()

TypeError: cannot unpack non-iterable NoneType object